# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
!pip install tqdm

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [3]:
# Load API Credentials
with open('/Users/purvikansara/.secret/yelp_api.json') as f:
    creds = json.load(f)

In [4]:
creds.keys()

dict_keys(['Client-ID', 'API Key'])

In [21]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(creds['API Key'],timeout_s=5.0)
yelp

### Define Search Terms and File Paths

In [8]:
# set our API call parameters and filename before the first call
LOCATION = "Seattle, WA 98122"
TERM = "Coffee"

In [15]:
## Specify folder for saving data
FOLDER = "Data/"
os.makedirs(FOLDER,exist_ok = True)

In [16]:
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+"Seattle Coffee.json"

In [17]:
JSON_FILE

'Data/Seattle Coffee.json'

In [18]:
LOCATION.split(',')[0]

'Seattle'

In [19]:
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+ f"{LOCATION.split(',')[0]}-{TERM}.json"

In [20]:
JSON_FILE

'Data/Seattle-Coffee.json'

### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [23]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term=TERM,location = LOCATION)
type(results)


dict

In [24]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [26]:
type(results['businesses'])


list

In [27]:
len(results['businesses'])

20

In [37]:
results['businesses'][0]

{'id': 'yLZborRy-JRAKtTWBk9a9g',
 'alias': 'union-coffee-and-wine-seattle',
 'name': 'Union Coffee and Wine ',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/99K9k7mqUWBoCPCLIzTvJQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/union-coffee-and-wine-seattle?adjust_creative=r8Qlusnw5U_OXTmzDH90Ug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=r8Qlusnw5U_OXTmzDH90Ug',
 'review_count': 96,
 'categories': [{'alias': 'coffee', 'title': 'Coffee & Tea'},
  {'alias': 'wine_bars', 'title': 'Wine Bars'}],
 'rating': 4.5,
 'coordinates': {'latitude': 47.61274, 'longitude': -122.30103},
 'transactions': ['delivery'],
 'price': '$',
 'location': {'address1': '2407 E Union St',
  'address2': 'Ste B',
  'address3': None,
  'city': 'Seattle',
  'zip_code': '98122',
  'country': 'US',
  'state': 'WA',
  'display_address': ['2407 E Union St', 'Ste B', 'Seattle, WA 98122']},
 'phone': '+17066314474',
 'display_phone': '(706) 631-4474',
 'distance': 406.41569

In [38]:
results['total']

1500

In [29]:
results['region']

{'center': {'longitude': -122.29568481445312, 'latitude': 47.61309534151991}}

In [30]:
# Lets save this into Pandas dataframe
pd.DataFrame(results['businesses'])


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,yLZborRy-JRAKtTWBk9a9g,union-coffee-and-wine-seattle,Union Coffee and Wine,https://s3-media2.fl.yelpcdn.com/bphoto/99K9k7...,False,https://www.yelp.com/biz/union-coffee-and-wine...,96,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 47.61274, 'longitude': -122.30103}",[delivery],$,"{'address1': '2407 E Union St', 'address2': 'S...",+17066314474,(706) 631-4474,406.415695
1,mUtrinuyznLic5wYc09LeA,arosa-cafe-seattle-6,Arosa Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/Sz1XcR...,False,https://www.yelp.com/biz/arosa-cafe-seattle-6?...,105,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 47.62622241003878, 'longitude': -...","[delivery, pickup]",$,"{'address1': '3121 E Madison St', 'address2': ...",+12064050605,(206) 405-0605,1487.954980
2,yCLUXtS6a1xMicPSsmdTfA,squirrel-chops-seattle,Squirrel Chops,https://s3-media2.fl.yelpcdn.com/bphoto/0fA2eb...,False,https://www.yelp.com/biz/squirrel-chops-seattl...,90,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 47.6128151, 'longitude': -122.303...",[],$,"{'address1': '2201 E Union St', 'address2': ''...",+12064857017,(206) 485-7017,572.723877
3,9TnUll8eOc5FREIQqrcrFg,verite-coffee-seattle,Verite Coffee,https://s3-media4.fl.yelpcdn.com/bphoto/SaSFeE...,False,https://www.yelp.com/biz/verite-coffee-seattle...,38,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.0,"{'latitude': 47.61175, 'longitude': -122.28953}",[],$,"{'address1': '1101 34th Ave', 'address2': '', ...",+12067094497,(206) 709-4497,483.915129
4,ebglfaWBLj-z7yqpG3BcsQ,golden-wheat-bakery-seattle,Golden Wheat Bakery,https://s3-media1.fl.yelpcdn.com/bphoto/YasEcz...,False,https://www.yelp.com/biz/golden-wheat-bakery-s...,128,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.5,"{'latitude': 47.608213, 'longitude': -122.294437}",[delivery],$,"{'address1': '2908 E Cherry St', 'address2': '...",+12063255055,(206) 325-5055,550.890653
5,Q8Nlx_FeZ4LW0kdbnHoyrw,qed-coffee-seattle,QED Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/HwagNI...,False,https://www.yelp.com/biz/qed-coffee-seattle?ad...,104,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 47.5896816513796, 'longitude': -1...",[delivery],$$,"{'address1': '1418 31st Ave S', 'address2': ''...",+12062357678,(206) 235-7678,2615.675293
6,eHM1er15ptmC6Q21-9ZteA,central-cafe-and-juice-bar-seattle,Central Cafe & Juice Bar,https://s3-media2.fl.yelpcdn.com/bphoto/41ZaCQ...,False,https://www.yelp.com/biz/central-cafe-and-juic...,60,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",5.0,"{'latitude': 47.60782, 'longitude': -122.29946}","[delivery, pickup]",$,"{'address1': '2509 E Cherry St', 'address2': N...",,,651.324295
7,e7uwwJKDamfG7psm5hZdtw,good-neighbor-cafe-seattle-2,Good Neighbor Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/_4ZQZx...,False,https://www.yelp.com/biz/good-neighbor-cafe-se...,44,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 47.60321, 'longitude': -122.30742}","[delivery, pickup]",NaN,"{'address1': '166 19th Ave', 'address2': 'Ste ...",+12063287002,(206) 328-7002,1405.654043
8,2ZrovOWUNfzpSJlx-7DYCw,hello-em-viet-coffee-and-roastery-seattle-2,Hello Em Viet Coffee & Roastery,https://s3-media2.fl.yelpcdn.com/bphoto/Wua0EA...,False,https://www.yelp.com/biz/hello-em-viet-coffee-...,235,"[{'alias': 'coffeeroasteries', 'title': 'Coffe...",4.5,"{'latitude': 47.59732, 'longitude': -122.31544}",[],$,"{'address1': '1227 S Weller St', 'address2': '...",,,2296.777516
9,SALUHYOpT9mk_GHfGzUKXw,temple-pastries-seattle-2,Temple Pastries,https://s3-media1.fl.yelpcdn.com/bphoto/Xf1Dld...,False,https://www.yelp.com/biz/temple-pastries-seatt...,100,"[{'alias': 'bakeries', 'title': 'Bakeries'}]",4.5,"{'latitude': 47.599309353938516, 'longitude': ...",[delivery],NaN,"{'address1': '2524 S Jackson St', 'address2': ...",+12064084131,(206) 408-4131,154

### Check if Json File exists and Create it if it doesn't

In [31]:
os.path.isfile(JSON_FILE)

False

In [32]:
## Check if JSON_FILE exists
if os.path.isfile(JSON_FILE)==False:
## If it does not exist: 
    print('The file does not exist.Creating empty file')    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump(results['businesses'],f)
        
## If it exists, inform user
else:
    print('File already exists.')

The file does not exist.Creating empty file


### Load JSON FIle and account for previous results

In [33]:
## Load previous results and use len of results for offset
prev_df = pd.read_json(JSON_FILE)
## set offset based on previous results
n_results = len(prev_df)
n_results

20

In [34]:
## How many results tottal

- Where is the actual data we want to save?

In [35]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [36]:
# Use math.ceil to round up for the total number of pages of results.
import math
n_pages= math.ceil(results['total']/results_per_page)
n_pages

75

In [39]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
        
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE) as f:
            prev_results = json.load(f)
            
        ## set offset based on previous results
        n_results = len(prev_results)
        
        ## save number of results for to use as offset  ## use n_results as the OFFSET 
        results = yelp.search_query(term =TERM,location=LOCATION, offset=n_results,limit=5)
 

        ## append new results and save to file
        
        prev_results.extend(results['businesses'])
        with open(JSON_FILE,'w') as f:
            json.dump(prev_results,f)
            
    ## What to do if we get an error/exception.
        


  0%|          | 0/75 [00:00<?, ?it/s]

## Open the Final JSON File with Pandas

In [45]:
df = pd.read_json(JSON_FILE)

In [48]:
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,yLZborRy-JRAKtTWBk9a9g,union-coffee-and-wine-seattle,Union Coffee and Wine,https://s3-media2.fl.yelpcdn.com/bphoto/99K9k7...,False,https://www.yelp.com/biz/union-coffee-and-wine...,96,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 47.61274, 'longitude': -122.30103}",[delivery],$,"{'address1': '2407 E Union St', 'address2': 'S...",+17066314474,(706) 631-4474,406.415695
1,mUtrinuyznLic5wYc09LeA,arosa-cafe-seattle-6,Arosa Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/Sz1XcR...,False,https://www.yelp.com/biz/arosa-cafe-seattle-6?...,105,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 47.62622241003878, 'longitude': -...","[delivery, pickup]",$,"{'address1': '3121 E Madison St', 'address2': ...",+12064050605,(206) 405-0605,1487.954980
2,yCLUXtS6a1xMicPSsmdTfA,squirrel-chops-seattle,Squirrel Chops,https://s3-media2.fl.yelpcdn.com/bphoto/0fA2eb...,False,https://www.yelp.com/biz/squirrel-chops-seattl...,90,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 47.6128151, 'longitude': -122.303...",[],$,"{'address1': '2201 E Union St', 'address2': ''...",+12064857017,(206) 485-7017,572.723877
3,9TnUll8eOc5FREIQqrcrFg,verite-coffee-seattle,Verite Coffee,https://s3-media4.fl.yelpcdn.com/bphoto/SaSFeE...,False,https://www.yelp.com/biz/verite-coffee-seattle...,38,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.0,"{'latitude': 47.61175, 'longitude': -122.28953}",[],$,"{'address1': '1101 34th Ave', 'address2': '', ...",+12067094497,(206) 709-4497,483.915129
4,ebglfaWBLj-z7yqpG3BcsQ,golden-wheat-bakery-seattle,Golden Wheat Bakery,https://s3-media1.fl.yelpcdn.com/bphoto/YasEcz...,False,https://www.yelp.com/biz/golden-wheat-bakery-s...,128,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.5,"{'latitude': 47.608213, 'longitude': -122.294437}",[delivery],$,"{'address1': '2908 E Cherry St', 'address2': '...",+12063255055,(206) 325-5055,550.890653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,E-afYN8HfodXEnG-u2X2ww,café-frieda-seattle-3,Café Frieda,https://s3-media3.fl.yelpcdn.com/bphoto/TptHPP...,False,https://www.yelp.com/biz/caf%C3%A9-frieda-seat...,18,"[{'alias': 'cafes', 'title': 'Cafes'}]",4.0,"{'latitude': 47.60709, 'longitude': -122.32416}",[],$$,"{'address1': '704 Terry Ave', 'address2': '', ...",+12066229250,(206) 622-9250,2234.568629
391,mMMCajfv-XQ9PzzUds04vg,cafe-hitchcock-seattle-2,Cafe Hitchcock,https://s3-media1.fl.yelpcdn.com/bphoto/SjjNZD...,False,https://www.yelp.com/biz/cafe-hitchcock-seattl...,79,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"{'latitude': 47.60374, 'longitude': -122.33505}",[delivery],$$,"{'address1': '818 1st Ave', 'address2': '', 'a...",,,3135.706843
392,ug_QHgsMXDO16_lSs6ywRQ,pacific-cafe-hong-kong-kitchen-seattle-5,Pacific Cafe Hong Kong Kitchen,https://s3-media2.fl.yelpcdn.com/bphoto/3TjbZN...,False,https://www.yelp.com/biz/pacific-cafe-hong-kon...,276,"[{'alias': 'desserts', 'title': 'Desserts'}, {...",4.0,"{'latitude': 47.59865, 'longitude': -122.32749}","[delivery, pickup]",$$,"{'address1': '416 5th Ave S', 'address2': '', ...",+12066820908,(206) 682-0908,2868.345941
393,bV-WWwIEKDZQsHckqq1z2A,uptown-espresso-belltown-seattle,Uptown Espresso - Belltown,https://s3-media2.fl.yelpcdn.com/bphoto/22xvOH...,False,https://www.yelp.com/biz/uptown-espresso-bellt...,285,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.0,"{'latitude': 47.6169628, 'longitude': -122.346...",[delivery],$,"{'address1': '2504 4th Ave', 'address2': '', '...",+12064411084,(206) 441-1084,3816.285151


In [49]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Seattle-Coffee.csv.gz'

In [47]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression='gzip', index=False)


## Bonus: compare filesize with os module's `os.path.getsize`

In [50]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 382,742 Bytes
CSV.GZ FILE: 53,844 Bytes
the csv.gz is 7.108350048287646 times smaller!


## Next Class: Processing the Results and Mapping 